In [1]:
# Import libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dt_ids7_export_utils as bh_utils

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Remove unnessecary columns:
df_ids7 = bh_utils.remove_unnecessary_columns(df_ids7, True)
# Check for missing time data:
df_ids7 = bh_utils.filter_NaT(df_ids7, True)
# Check for cancelled procedures:
df_ids7 = bh_utils.filter_cancelled(df_ids7, True)
# Check for non-human subjects (e.g. phantoms, animals, etc.):
df_ids7 = bh_utils.filter_phantom_etc(df_ids7, True)
# Check for invalid accession numbers:
df_ids7 = bh_utils.check_accession_format(df_ids7, True)
# Check whether an accession number is in the DoseTrack data:
df_ids7 = bh_utils.check_accession_ids7_vs_dt(df_ids7, df_dt, True)
# Check whether examination has duplicated accession numbers that should be overwritten:
df_ids7 = bh_utils.overwrite_duplicated_accession_numbers(df_ids7, df_dt, True)

Dropping unnecessary column: Prioritet- og lesemerkeikon
Dropping unnecessary column: Lagt til i demonstrasjon-ikon
Dropping unnecessary column: Status
Number of rows with NaT in the column "Bestilt dato og tidspunkt": 1
Number of cancelled procedures: 393
Number of non-human subjects: 7
Number of rows with invalid accession number: 0
Number of accession numbers in IDS7: 2218
Number of accession numbers in IDS7 not in DoseTrack: 136
Inserted accession number: NRRH000008244732 for patient: PAS0607, time: 2023-05-18 13:00:00, accession numbers: ['NRRH000008244733' 'NRRH000008244732']
Inserted accession number: NRRH000007967678 for patient: PAS1790, time: 2023-01-03 07:00:00, accession numbers: ['NRRH000007967678' 'NRRH000007967670']
The accession numbers have been changed. Running check_accession_ids7_vs_dt
Number of accession numbers in IDS7: 2216
Number of accession numbers in IDS7 not in DoseTrack: 134


In [2]:
# Check for systematic missing data in dosetrack.
missing = df_ids7[df_ids7['Henvisning_i_dt'] == False]
# Make a 


In [ ]:
# For the future:
# Make a function that plots a histogram with one bar for each week.
# The height of the bar represents the number of procedures for that lab for each week.
# The bar is split into two colors, green for procedures with a dose in DT, red for procedures without a dose in DT.